This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## The Transformer architecture

### Understanding self-attention

#### Generalized self-attention: the query-key-value model

### Multi-head attention

### The Transformer encoder

**Getting the data**

In [0]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

**Preparing the data**

In [1]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category, exist_ok=True)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

2025-05-13 17:42:55.933335: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747138375.951404  203116 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747138375.956853  203116 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 17:42:55.975194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 8192 files belonging to 2 classes.


I0000 00:00:1747138378.031737  203116 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8472 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:07:00.0, compute capability: 6.1


Found 16808 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**Vectorizing the data**

In [2]:
from tensorflow.keras import layers

max_length = 300
max_tokens = 10000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

2025-05-13 17:43:11.400406: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


**Transformer encoder implemented as a subclassed `Layer`**

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

**Using the Transformer encoder for text classification**

In [4]:
vocab_size = 10000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,104,033 (11.84 MB)

 Trainable params: 3,104,033 (11.84 MB)

 Non-trainable params: 0 (0.00 B)

**Training and evaluating the Transformer encoder based model**

In [5]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


I0000 00:00:1747138428.811144  203339 service.cc:148] XLA service 0x74921c004fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747138428.811187  203339 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
2025-05-13 17:43:48.853097: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747138429.212254  203339 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/256 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.5091 - loss: 2.9986

I0000 00:00:1747138433.513626  203339 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


256/256 ━━━━━━━━━━━━━━━━━━━━ 17s 45ms/step - accuracy: 0.5439 - loss: 1.0056 - val_accuracy: 0.7855 - val_loss: 0.4614
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.7887 - loss: 0.4433 - val_accuracy: 0.8262 - val_loss: 0.3873
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.8333 - loss: 0.3783 - val_accuracy: 0.8346 - val_loss: 0.3688
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 10s 41ms/step - accuracy: 0.8629 - loss: 0.3215 - val_accuracy: 0.8413 - val_loss: 0.3581
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - accuracy: 0.8723 - loss: 0.2918 - val_accuracy: 0.8305 - val_loss: 0.3959
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.8953 - loss: 0.2564 - val_accuracy: 0.8485 - val_loss: 0.3588
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9118 - loss: 0.2200 - val_accuracy: 0.8115 - val_loss: 0.4776
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.9231 - loss: 0.1929 - val_accurac

/store/venv/da5401/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8429 - loss: 0.3549
Test acc: 0.843


#### Using positional encoding to re-inject order information

**Implementing positional embedding as a subclassed layer**

In [8]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    #def compute_mask(self, inputs, mask=None):
    #    return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

#### Putting it all together: A text-classification Transformer

**Combining the Transformer encoder with positional embedding**

In [9]:
vocab_size = 10000
sequence_length = 200
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding_1          │ (None, None, 256)      │     2,611,200 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_1           │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_1          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,155,233 (12.04 MB)

 Trainable params: 3,155,233 (12.04 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


2025-05-13 18:11:49.611020: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at scatter_nd_op.cc:176 : INVALID_ARGUMENT: indices[299] = [299] does not index into shape [200]
2025-05-13 18:11:49.611520: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at scatter_nd_op.cc:176 : INVALID_ARGUMENT: indices[299] = [299] does not index into shape [200]
2025-05-13 18:11:49.612536: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at scatter_nd_op.cc:176 : INVALID_ARGUMENT: indices[299] = [299] does not index into shape [200]


256/256 ━━━━━━━━━━━━━━━━━━━━ 16s 36ms/step - accuracy: 0.5589 - loss: 0.9789 - val_accuracy: 0.6727 - val_loss: 0.5928
Epoch 2/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7560 - loss: 0.5071 - val_accuracy: 0.7696 - val_loss: 0.4729
Epoch 3/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.8065 - loss: 0.4135 - val_accuracy: 0.8104 - val_loss: 0.4158
Epoch 4/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.8486 - loss: 0.3521 - val_accuracy: 0.8061 - val_loss: 0.4254
Epoch 5/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - accuracy: 0.8679 - loss: 0.3132 - val_accuracy: 0.8273 - val_loss: 0.3911
Epoch 6/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.8910 - loss: 0.2650 - val_accuracy: 0.8238 - val_loss: 0.4054
Epoch 7/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9092 - loss: 0.2256 - val_accuracy: 0.8307 - val_loss: 0.3982
Epoch 8/20
256/256 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9259 - loss: 0.1884 - val_accuracy: 0.80

/store/venv/da5401/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'positional_embedding_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/store/venv/da5401/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8293 - loss: 0.3805
Test acc: 0.826


### When to use sequence models over bag-of-words models?